# Alzheimer GWAS Thesis Project - Chrysania Lim i3L

this notebook contains the backup codes for all hail and variantspark operations, including:
- data set up: *completed*
- logistic regression (diagnosis): *completed*
- random forest (diagnosis): *in progress*
- linear regression (APOE, age): *completed*

need to add code to check for space complexity and time efficiency

**DO NOT RUN ANYTHING ON THIS NOTEBOOK**
running any cell could make the notebook unsavable, idk how, idk why, idk which code specifically because it seems like its random every time.

~~i swear hail is cursed~~ 

## Data set up

In [ ]:
import hail as hl
import varspark.hail as vshl
vshl.init()

In [ ]:
from hail.plot import show
from pprint import pprint
# hl.plot.output_notebook()

In [ ]:
# vcf file was converted from plink
data = hl.import_vcf('alzheimer_vcf_2.vcf', contig_recoding={'23': 'X', '24' : 'Y', '25': 'MT'})

In [ ]:
data.describe()

In [ ]:
# label & data format covariate file
labels = hl.import_table('covariates_hail3.csv', delimiter=';',
                        types={'FID_IID':'str', 'FID':'str', 'IID':'str', 'Diagnosis':'float64', 'Age':'float64', 'APOE':'float64', 'Region':'float64', 'PMI':'float64', 'Site':'float64', 'HybridizationDate':'float64'},
                        key='FID_IID')

In [ ]:
labels.describe()

In [ ]:
mt = data.annotate_cols(labels = labels[data.s])
mt.describe()

In [ ]:
# check if data is complete
mt = mt.annotate_cols(
    can_be_used_for_regression = (hl.is_defined(mt.labels.Diagnosis) &
                                  hl.is_defined(mt.labels.Age) &
                                  hl.is_defined(mt.labels.APOE) &
                                  hl.is_defined(mt.labels.Region) &
                                  hl.is_defined(mt.labels.PMI) &
                                  hl.is_defined(mt.labels.Site) &
                                  hl.is_defined(mt.labels.HybridizationDate)))
mt.can_be_used_for_regression.show() # WARNING! running this will make notebook unsavable

In [ ]:
# check number of variants and samples
mt.count()

# note: removed 778 SNPs in chr 3,4,19,MT due to 'not within the range of GRCh37'
# note: all of chrMT SNPs are removed

In [ ]:
# check vcf file
mt.show(n_cols=3)

## Logistic Regression - Hail

Logistic regression based on Diagnosis (case/control)

In [ ]:
# logistic regression: Diagnosis
gwas_logreg_diagnosis = hl.logistic_regression_rows(test='score',
                                y=mt.labels.Diagnosis,
                                 x=mt.GT.n_alt_alleles(),
                                 covariates=[1.0, mt.labels.Age, mt.labels.APOE],
                                 pass_through=[mt.rsid])

In [ ]:
manhattan_logreg_diagnosis = hl.plot.manhattan(gwas_logreg_diagnosis.p_value, hover_fields=dict(rs=gwas_logreg_diagnosis.rsid))
show(manhattan_logreg_diagnosis)

In [ ]:
hl.plot.output_notebook() # WARNING! running this will make notebook unsavable

p = hl.plot.manhattan(gwas_logreg_diagnosis.p_value, hover_fields=dict(rs=gwas_logreg_diagnosis.rsid))
show(p)

## Random Forest - VariantSpark

In [ ]:
rf_model = vshl.random_forest_model(y=mt.labels.Diagnosis,
                    x=mt.GT.n_alt_alleles(), 
                    covariates={'Age':mt.labels.Age, 'APOE':mt.labels.APOE})
rf_model.fit_trees(500, 100)

In [ ]:
# Capture variant importances
print(rf_model.oob_error())
impTable = rf_model.variable_importance()
impTable.show(3)

In [ ]:
# Show the covariates importances
covImpTable = rf_model.covariate_importance()
covImpTable.show(4)

In [ ]:
# Join hail and VariantSpark results (this is only needed here to get the RSID's)
gwas_with_imp = gwas.join(impTable)

In [ ]:
import varspark.hail.plot as vshlplt

p = vshlplt.manhattan_imp(gwas_with_imp.importance, 
                            hover_fields=dict(ri=gwas_with_imp.rsid),
                            significance_line = None)
show(p)

In [ ]:
# Compare logistc regression values vs. rf importance
p = hl.plot.scatter(x=-hl.log10(gwas_with_imp.p_value),
                    y=gwas_with_imp.importance, 
                    xlabel = '-log10(p-value)',
                    ylabel = 'gini importance',
                    hover_fields=dict(rs=gwas_with_imp.rsid, loc=gwas_with_imp.locus))
show(p)

## Linear Regression - Hail

Linear regression based on APOE (1/2)

In [ ]:
# Linear regression: APOE
gwas_linreg_APOE = hl.linear_regression_rows(y=mt.labels.APOE,
                                 x=mt.GT.n_alt_alleles(),
                                 covariates=[1.0, mt.labels.Diagnosis, mt.labels.Age],
                                 pass_through=[mt.rsid])

In [ ]:
gwas_linreg_APOE.show()

In [ ]:
linreg_APOE_plot = hl.plot.manhattan(gwas_linreg_APOE.p_value, hover_fields=dict(rs=gwas_linreg_APOE.rsid))
show(linreg_APOE_plot)

Linear regression based on Age (2/2)

In [ ]:
# Linear regression: Age
gwas_linreg_Age = hl.linear_regression_rows(y=mt.labels.Age,
                                 x=mt.GT.n_alt_alleles(),
                                 covariates=[1.0, mt.labels.Diagnosis, mt.labels.APOE],
                                 pass_through=[mt.rsid])

In [ ]:
gwas_linreg_Age.show()

In [ ]:
linreg_Age_plot = hl.plot.manhattan(gwas_linreg_Age.p_value, hover_fields=dict(rs=gwas_linreg_Age.rsid))
show(linreg_Age_plot)

--- End of jupyter notebook ---